<a href="https://colab.research.google.com/github/aknip/llm-benchmark/blob/main/llm_simple_benchmark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai litellm pandas openpyxl

In [9]:
import json
import os
from getpass import getpass
import psutil
import requests
IN_NOTEBOOK = any(["jupyter-notebook" in i for i in psutil.Process().parent().cmdline()])
if IN_NOTEBOOK:
  CREDS = json.loads(getpass("Secrets (JSON string): "))
  os.environ['CREDS'] = json.dumps(CREDS)
  CREDS = json.loads(os.getenv('CREDS'))

Secrets (JSON string): ··········


In [19]:
from litellm import completion
import openai
os.environ["OPENAI_API_KEY"] = CREDS['OpenAI']['v1']['credential'] # my key
os.environ["TOGETHERAI_API_KEY"] = CREDS['together-ai']['key']['credential']

# Call via OpenAI

In [25]:
import openai
import os

client = openai.OpenAI(
  api_key=os.environ.get("TOGETHERAI_API_KEY"),
  base_url='https://api.together.xyz',
)

chat_completion = client.chat.completions.create(
  messages=[
    {
      "role": "user",
      "content": "Tell me about San Francisco",
    }
  ],
  model="mistralai/Mixtral-8x7B-Instruct-v0.1",
  max_tokens=1024
)

print(chat_completion.choices[0].message.content)

San Francisco is a vibrant and culturally diverse city located in northern California, USA. It is known for its steep rolling hills, iconic landmarks, and cool, foggy weather. Here are some key facts and points of interest about San Francisco:

1. History: San Francisco was founded in 1776 by Spanish colonists and was originally a small mission and military outpost. It grew rapidly during the California Gold Rush in the mid-1800s and became a major commercial and cultural center.
2. Geography: San Francisco is situated on the tip of a peninsula, surrounded by the San Francisco Bay and the Pacific Ocean. It covers an area of about 47 square miles and has a population of approximately 875,000 people.
3. Landmarks: San Francisco is home to many iconic landmarks, including the Golden Gate Bridge, Alcatraz Island, Fisherman's Wharf, Lombard Street, and Chinatown.
4. Culture: San Francisco is known for its vibrant cultural scene, with world-class museums, theaters, and music venues. It is al

# Call via litellm

In [15]:
response = completion(
  model="gpt-3.5-turbo",
  messages=[{ "content": "Hello, how are you?","role": "user"}]
)
print(response.choices[0].message.content)
print(response)

Hello! I'm an AI language model, so I don't have feelings, but I'm here to help and answer any questions you may have. How can I assist you today?
ModelResponse(id='chatcmpl-8aJq4ZzYqZx9RpZZISWALpdBwgCX0', choices=[Choices(finish_reason='stop', index=0, message=Message(content="Hello! I'm an AI language model, so I don't have feelings, but I'm here to help and answer any questions you may have. How can I assist you today?", role='assistant'))], created=1703666759, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=Usage(completion_tokens=37, prompt_tokens=13, total_tokens=50), _response_ms=2601.7450000000003)


In [24]:
response = completion(
  #model="together_ai/togethercomputer/Llama-2-7B-32K-Instruct",
  model="together_ai/mistralai/Mixtral-8x7B-Instruct-v0.1",
  messages=[{ "content": "Hello, how are you?","role": "user"}]
)
print(response.choices[0].message.content)
print(response)

Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to assist you with any questions you have. Is there something specific you would like to know or talk about?
ModelResponse(id='chatcmpl-060eab6b-3cfb-45dd-ad25-9a2ff02dfbfe', choices=[Choices(finish_reason='stop', index=0, message=Message(content="Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to assist you with any questions you have. Is there something specific you would like to know or talk about?", role='assistant'))], created=1703667116, model='mistralai/Mixtral-8x7B-Instruct-v0.1', object='chat.completion', system_fingerprint=None, usage=Usage(prompt_tokens=12, completion_tokens=44, total_tokens=56), _response_ms=2404.737)


# 1. Create Excel file

In [ ]:
import pandas as pd
import openpyxl
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows

# see https://openpyxl.readthedocs.io/en/2.4/pandas.html

# Create a Pandas dataframe from the data.
df = pd.DataFrame({'Name': ['Miller', 'Adams', 'Smith'],
                   'Prompt': ['Write something', 'Do different things', 'Summarize it'], })

# Create a Pandas Excel writer using openpyxl as the engine.
writer = pd.ExcelWriter('llm_benchmark.xlsx', engine='openpyxl')

# Convert the dataframe to an XlsxWriter Excel object.
df.to_excel(writer, sheet_name='Benchmark', index=True)

# Get the xlsxwriter objects from the dataframe writer object.
workbook  = writer.book
worksheet = writer.sheets['Benchmark']

# Save the Excel file.
workbook.save('llm_benchmark.xlsx')

# Alternate approach without ExcelWriter:
# workbook = Workbook()
# worksheet = workbook.active
# for row in dataframe_to_rows(df, index=True, header=True):
#     if row != [None]:
#       worksheet.append(row)

# 2. Read Excel file as dataframe

In [ ]:
import pandas as pd
#import xlrd

df = pd.read_excel('llm_benchmark.xlsx', sheet_name='Benchmark') # parse_dates=['date'] # dtype={'column_name': float}
#print(df.head())

print("Given Dataframe :\n", df)

print("\nIterating 1:")
for i in df.index:
    print(i, df['Name'][i],df['Prompt'][i])

print("\nIterating 2:")
for index, row in df.iterrows():
    name = row['Name']
    prompt = row['Prompt']
    print(f"{index}: {name}, {prompt}")

# 3. Updata data and Excel file

In [ ]:
df.at[1, 'Name'] = 'Name-Updated'
df.at[2, 'Prompt'] = 'Prompt-Updated'
print(df.head())

In [ ]:
import pandas as pd
import openpyxl
from openpyxl import Workbook
from openpyxl import load_workbook
workbook = load_workbook(filename = 'llm_benchmark.xlsx')
worksheet = workbook.active

print("Updating Excel file:")
for index, row in df.iterrows():
    name = row['Name']
    prompt = row['Prompt']
    print(f"{index}: {name}, {prompt}")
    d = worksheet.cell(row=index+2, column=2, value=name)
    d = worksheet.cell(row=index+2, column=3, value=prompt)

# Alternative: delete data rows and append dataframe to worksheet
# continuously delete row 2 until there is only first row (header row) is left over
# while(worksheet.max_row > 1):
#     worksheet.delete_rows(2) # removes the row 2
# append dataframe to worksheet
# for row in dataframe_to_rows(df, index=False, header=False):
#     if row != [None]:
#        worksheet.append(row)

# Save the Excel file.
workbook.save('llm_benchmark.xlsx')